# Circus Charlie

## Librerias a utilizar

Entorno de emulación retro-gym

In [1]:
import retro

Opencv es una librería de visión artificial, utilizada para redimensionar y transformar imágenes

In [2]:
import cv2

Numpy utilizado para transformar frames a matrices

In [3]:
import numpy as np

Pickle destinado a dumpear variables importantes

In [4]:
import pickle

Neat es una librería destinada a la configuración de redes neuronales con algoritmos genéticos en base a parámetros entregados de forma sencilla

In [5]:
import neat

## Descripción del entorno
Mediante el siguiente comando se inicializa el entorno retro-gym

In [6]:
env = retro.make(game='CircusCharlie-Nes',state='CircusCharlie.level2.1live',record=True)

El entorno obtenido es en base a el videojuego seleccionado y el estado inicial del cual comenzar un generación

## Implementación

### Parametros de conversión.
Obtiene el ancho y alto que tendra cada frame

In [7]:
width,height,_ = env.observation_space.shape

In [8]:
print(width,'x',height,'pixels')

224 x 240 pixels


Reduce ambos a una octava parte, para luego redimensionar la imagen.

In [9]:
width = int(width/8)
height = int(height/8)
print(width,'x',height,'pixels')

28 x 30 pixels


### Conversión de imagen
La siguiente función recibe como parametro un frame y retorna su conversión a un arreglo unidimensional

In [10]:
def frame_to_array(ob):
    # Redimensiona el frame a una octava parte del mismo
    ob = cv2.resize(ob,(width,height))
    # Transforma el frame a escala de grises
    # El color no es importante solo significa más ruido para analizar
    ob = cv2.cvtColor(ob, cv2.COLOR_BGR2GRAY)
    # Transforma el frame cv2 a matriz numpy
    ob = np.reshape(ob,(width,height))
    # Transforma frame de una matriz bidimensional a un arreglo unidimensional
    return ob.flatten()

### Función de fitness
Evalua los parametros obtenidos al realizar una acción en la función de fitness actualizando este valor para el genoma actual

In [11]:
def fitness_function(current_score,rew,jump_score_variation,backward_walk):
    current_score += (rew+(jump_score_variation)*50-backward_walk*0.1)
    return current_score

Los parametros que utiliza son el escore actual acumulado, el reward obtenido de la acción anterior realizada por el genoma (1 ó 0), los saltos sobre los simios producto de la acción anterior y la penalización por retroceder.

### Funión de evaluación
La función a continuación recibe la cantidad de genomas a ejecutar y los parametros ya definidos para la red neuronal

In [12]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        # Primer frame
        ob = env.reset()
        # Inicializa la red neuronal
        net = neat.nn.RecurrentNetwork.create(genome,config)
        # Mayor puntaje obtenido para la generación actual
        current_max_score = 0
        # Puntaje obtenido por el genome actual
        current_score = 0
        # Contador de la cantidad de movimientos
        # realizados por el gnome con mayor puntaje
        counter = 0
        # Valor de estado actual, indica si la partida acaba
        done = False
        # Numero de saltos iniciales ya realizados hasta la etapa 2
        # Ahora en la etapa 2 se consideran saltos sobre simios
        # Inicia en 132 por movimientos realizados en la etapa 1
        prev_jump_score = 132
        # Mientras la partida no acabe
        while not done:
            # env.render()
            # Transforma el frame anterior a un arreglo unidimensional
            oned_image = frame_to_array(ob)
            # Recibe el output desde la red neuronal a partir del frame entregado
            neuralnet_output = net.activate(oned_image)
            # Movimiento de salida redondeado a su valor entero mas proximo 0 o 1.
            neuralnet_output = [round(button,0) for button in neuralnet_output]
            # Ejecuta acción obtenida anteriormente
            # Obteniendo un nuevo frame ob
            # un nuevo reward dependiendo del movimiento realizado
            # y un nuevo indicador de done para saber el estado de Charlie
            ob, rew, done, info = env.step(neuralnet_output)

            # Saltos sobre simios actuales
            actual_jump_score = info.get("jump_score")
            # Variación de saltos sobre simios
            if actual_jump_score > prev_jump_score:
                jump_score_variation = actual_jump_score-prev_jump_score
                # Actualiza el valor de saltos sobre simios
                prev_jump_score = actual_jump_score
            else:
                jump_score_variation = 0
            
            # Verifica si Charlie dio un paso hacia atrás (el muy cobarde)
            if neuralnet_output[6]:
                backward_walk = 1
            else:
                backward_walk = 0
            
            # Score actual acumulado
            current_score = fitness_function(current_score,rew,jump_score_variation,backward_walk)

            # Si el puntaje obtenido por el genome actual
            # es mayor que el maximo de toda la generación
            if current_score>current_max_score:
                # El puntaje obtenido pasa a ser el nuevo maximo
                current_max_score = current_score
                # Reinicia el contador de pasos para llegar a este
                counter = 0
            # De no cumplirse
            else:
                # Suma al contador de pasos hasta llegar a este
                counter+=1

            # Si la partida acaba (mono colisiona con Charlie)
            if done:
                # Marca la partida como terminada
                done = True 
                # Despliega el numero del genome con su puntaje obtenido
                print(genome_id,current_score)

            # Actualiza el puntaje del genoma por cada iteración de este
            genome.fitness = current_score

Para este caso la cantidad de genomes por generación estara estipulada en el fichero de configuración que es almacenado en la variable config, por tener las funcionalidades de un algoritmo genetico el tamaño de la población no debe ser muy bajo.

### main

In [14]:
if __name__ == "__main__":
    # Inicializa la configuración de la red neuronal y las caracteristicas de los genomas
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,neat.DefaultSpeciesSet, neat.DefaultStagnation,'config-feedforward')
    # Algorimo genetico de crecimiento poblacional
    p = neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    p.add_reporter(neat.StatisticsReporter())
    p.add_reporter(neat.Checkpointer(10))
    winner = p.run(eval_genomes)
    with open('winner.pkl', 'wb') as output:
        pickle.dump(winner, output, 1)


 ****** Running generation 0 ****** 

1 119.80000000000024
2 -78.29999999999983
3 -78.29999999999983
4 -83.69999999999952
5 -74.10000000000007
Population's average fitness: -38.92000 stdev: 79.41850
Best fitness: 119.80000 - size: (12, 5046) - species 1 - id 1
Average adjusted fitness: 0.220
Mean genetic distance 1.309, standard deviation 0.822
Population of 5 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     5    119.8    0.220     0
Total extinctions: 0
Generation time: 50.351 sec

 ****** Running generation 1 ****** 

1 119.80000000000024
5 -74.10000000000007
2 -78.29999999999983
6 52.0
7 119.80000000000024
Population's average fitness: 27.84000 stdev: 88.49232
Best fitness: 119.80000 - size: (12, 5046) - species 1 - id 1
Average adjusted fitness: 0.536
Mean genetic distance 0.643, standard deviation 0.265
Population of 5 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======

KeyboardInterrupt: 